In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import re

/Users/zenmaster/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [45]:
pd.set_option('display.max_rows',None)


In [28]:
pd.set_option('display.max_colwidth', None)

In [2]:
options = Options() 
driver = webdriver.Chrome(options=options)
driver.get('https://www.clasificadosonline.com/RealEstate.asp')

In [49]:
multi_check_box_toggle = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[1]')
multi_check_box_toggle.click()
time.sleep(2)

In [50]:
select_all = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[2]/div[1]/input')
select_all.click()
time.sleep(2)

In [51]:
time.sleep(10)
ver_listado = driver.find_element(By.XPATH,'//*[@id="BtnSearchListing"]')
ver_listado.click()

In [54]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')

In [ ]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')
order_by.click()
ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
ultimos_publicados.click()

In [12]:
list_cards = []

for i in range(1,16):
    table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
    table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
    table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    dictionary_cards = {}
    dictionary_cards_2 = {}
    try:
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text
        
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text

    except NoSuchElementException:
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
    try:
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards['Broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')

        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards_2['Broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')

    except NoSuchElementException:
        dictionary_cards['Broker'] = 'No Broker'
        dictionary_cards_2['Broker'] = 'No Broker'

    dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
    dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
    dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
    dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
    dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
    list_cards.append(dictionary_cards)

    dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
    dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
    dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
    dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
    dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
    list_cards.append(dictionary_cards_2)


In [ ]:
df = pd.json_normalize(list_cards)
df

In [ ]:
next_page = driver.find_element(By.XPATH,'//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
next_page.click()

In [5]:
last_modified = driver.execute_script("return document.lastModified;")

In [ ]:
last_modified

In [ ]:
import requests

# Send a GET request to the webpage
response = requests.get("https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4774562")  # Replace this URL with the URL of your webpage

# Check the Last-Modified header in the response
last_modified_header = response.headers.get("Last-Modified")

# Print the Last-Modified header
print("Last modified (from HTTP header):", last_modified_header)

In [ ]:
all_cards = []

# Infinite loop to navigate through pages
for x in range(3):
    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'

        try:
            dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
            dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
            dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            list_cards.append(dictionary_cards)
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'

        try:
            dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
            dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
            list_cards.append(dictionary_cards_2)
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        time.sleep(10)
        next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        next_page.click()
        time.sleep(7)  # Add delay to allow page to load
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)

# Save to CSV or process further
df.to_csv("classifieds_data.csv", index=False)

print("Data collection completed.")

In [24]:
def clean_real_estate_data(df):
    # Clean the 'Type' column by removing leading commas and whitespace
    df['type'] = df['type'].str.replace(r'^,\s*', '', regex=True).str.strip()

    # Clean the 'Broker' column using the combined logic
    def clean_broker(broker_name):
        try:
            # Remove any text starting with " - " followed by anything after it
            broker_name = re.sub(r'\s*-\s*.*$', '', broker_name)
            
            # Remove any text starting with "#" followed by digits
            broker_name = re.sub(r'\s*#\d+', '', broker_name)
            
            # Remove any text starting with "lic." followed by digits
            broker_name = re.sub(r'\s*lic\.\s*\d+', '', broker_name)
            
        except Exception as e:
            print(f"An error occurred: {e}")

        # Return the cleaned broker name
        return broker_name.strip()
    
    # Apply the clean_broker function to the 'Broker' column
    df['broker'] = df['broker'].apply(clean_broker)

    # Drop the 'Title' column if needed
    # df.drop(columns=['Title'], inplace=True)

    # Extract the number of bedrooms and bathrooms from the 'Rooms' column
    df[['bedrooms', 'bathrooms']] = df['rooms'].str.extract(r'(\d+)\s+Cuartos.*\|\s*(\d+)\s+Baños', expand=True)
    df[['bedrooms', 'bathrooms']] = df[['bedrooms', 'bathrooms']].fillna(0)

    df['location'] = df['location'].fillna('')

    # Split the 'Location' column by the comma into two columns
    df[['neighborhood', 'city']] = df['location'].str.split(',', n=1, expand=True)

    # Strip any leading or trailing whitespace from the new columns
    df['neighborhood'] = df['neighborhood'].str.strip()
    df['city'] = df['city'].str.strip()

    df['broker'] = df['broker'].str.replace('ClasificadosOnline', '').str.strip()

    # Check if the word from the name column in municipio is present in the city column within data
    df['city_name'] = df['city'].apply(lambda x: next((name for name in municipio['name'] if name in x), None))

    # Merge the dataframes on the extracted city names and the name column
    merged_df = pd.merge(df, municipio, left_on='city_name', right_on='name', how='left')

    # Add the region column to the original data dataframe
    df['region'] = merged_df['region']

    # Drop the temporary city_name column
    df.drop(columns=['city_name'], inplace=True)

    df['price'] = df['price'].str.replace('$','').str.replace(',','').astype(float)

    return df

In [25]:
municipio = pd.read_csv('PR_Municipios')

In [26]:
# Initialize an empty list to store all data
all_cards = []

# Initialize a page counter (optional, for debugging or tracking pages)
page_number = 1

# Infinite loop to navigate through pages
for x in range(3):

    dropdown_element = driver.find_element(By.XPATH, '//*[@id="jumpMenu"]')
    select = Select(dropdown_element)
    selected_option = select.first_selected_option

    print(f"Scraping page {page_number}...")

    if selected_option.text !=  'Ultimos Publicados (Más Recientes Primeros)':
        dropdown_element.click()
        time.sleep(2)
        ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
        ultimos_publicados.click()
        time.sleep(7)


    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        table_1_piclink = table_1 + f'> tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td > div > a > img'
        
        try:
            dictionary_cards['title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            
            # type validation, since a property can have two types
            try: 

                table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
                dictionary_cards['type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text

            except NoSuchElementException:
                dictionary_cards['type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text


            dictionary_cards['type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
            dictionary_cards['location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
            dictionary_cards['link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            
            if "MultipleSellers" in dictionary_cards['link']:
                dictionary_cards['broker'] = 'Multiple Sellers'

            else:

                # not all properties have brokers
                try:
                    dictionary_cards['broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards['broker'] = 'No Broker'


            dictionary_cards['piclink'] = driver.find_element(By.CSS_SELECTOR, table_1_piclink).get_attribute('src')

            list_cards.append(dictionary_cards)
            
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        table_2_piclink = table_2 + f'> tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td > div > a > img'

        try:
            dictionary_cards_2['title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            
            # type validation, since a property can have two types
            try:
                table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
                dictionary_cards_2['type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text
            except NoSuchElementException:
                dictionary_cards_2['type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            
            dictionary_cards_2['location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
            dictionary_cards_2['link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')

            if "MultipleSellers" in dictionary_cards_2['link']:
                dictionary_cards_2['broker'] = 'Multiple Sellers'

            else:
                try:
                    dictionary_cards_2['broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards_2['broker'] = 'No Broker'

            dictionary_cards_2['piclink'] = driver.find_element(By.CSS_SELECTOR, table_2_piclink).get_attribute('src')

            list_cards.append(dictionary_cards_2)
            
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        time.sleep(10)
        next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        next_page.click()
        time.sleep(7)  # Add delay to allow page to load
        page_number += 1
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)
cleaned_df = clean_real_estate_data(df)

Scraping page 1...
Scraping page 2...
Scraping page 3...


In [29]:
cleaned_df

,title,rooms,price,type,location,link,broker,picLink,bedrooms,bathrooms,neighborhood,city,region
0,Unique Investment Opportunity! Ocean ...,5 Cuartos | 5 Baños,650000.0,Casa,"Urbanizacion-Ocean Park , San Juan - Condado-Miramar",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4823723,Ocean Park de Enjolie Hernández Real Estate,https://imgcache.clasificadosonline.com//PP/FS/2024/9/16/09162024092552cfrtlo11.jpg,5,5,Urbanizacion-Ocean Park,San Juan - Condado-Miramar,metro
1,CANDELARIA- VENTA CASH,4 Cuartos | 2 Baños,124995.0,MultiFamiliar,"Barrio-Candelaria , Toa Baja",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4836690,Candelaria de My House Realty,https://imgcache.clasificadosonline.com//PP/FS/2024/12/23/12232024192547tvev223t.jpeg,4,2,Barrio-Candelaria,Toa Baja,metro
2,Apartamento Arroyo Beach (segundo pis...,2 Cuartos | 2 Baños,195000.0,Apartamento,"Condominio-Arroyo Beach Resort , Arroyo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4820277,Arroyo Beach Resort de Clary Gierbolini Real Estate,https://imgcache.clasificadosonline.com//PP/FS/2024/8/26/082620240742340y0x4czz.jpeg,2,2,Condominio-Arroyo Beach Resort,Arroyo,south
3,PH Condo en Haudimar Frente a la play...,3 Cuartos | 3 Baños,550000.0,Apartamento,"Barrio-Bajuras , Isabela",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838467,Bajuras de Graves Realty,https://imgcache.clasificadosonline.com//PP/FS/2025/1/12/01122025075728hauubw1p.jpeg,3,3,Barrio-Bajuras,Isabela,west
4,LAS VISTAS CLUSTER 6 2/2.5 SPECTACUL...,2 Cuartos | 2 Baños,750000.0,Apartamento,"Condominio-Las Vistas De Rio Mar , Río Grande",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4786708,Las Vistas De Rio Mar de FullHouse RE,https://imgcache.clasificadosonline.com//PP/FS/2024/1/18/01182024152843ou2h0c3s.jpg,2,2,Condominio-Las Vistas De Rio Mar,Río Grande,east
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,ESJ Towers Apto 350,1 Baños,275000.0,Apartamento,"Condominio-Esj Towers , Carolina - Isla Verde",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4817552,Esj Towers de PMI Puerto Rico,https://imgcache.clasificadosonline.com//PP/FS/2024/8/5/08052024103555pmjfm1ol.jpg,0,0,Condominio-Esj Towers,Carolina - Isla Verde,metro
86,"Terreno Urb. San Martín, Rio Piedras",,115000.0,Solar,"Urbanizacion-San Martin , San Juan - Río Piedras",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4822705,San Martin de PMI Puerto Rico,https://imgcache.clasificadosonline.com//PP/FS/2024/9/9/09092024142713ql1odnzg.jpg,0,0,Urbanizacion-San Martin,San Juan - Río Piedras,metro
87,casa tres cuartos dos banos cerca de ...,3 Cuartos | 2 Baños,75000.0,Casa,"Urbanizacion-Rafael Bermudez , Fajardo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4654045,Rafael Bermudez de JJ ENTERPRISES CORP,https://imgcache.clasificadosonline.com//PP/FS/2022/2/22/02222022125411vprexila.jpg,3,2,Urbanizacion-Rafael Bermudez,Fajardo,east
88,Casa multifamiliar con guarderia para...,4 Cuartos | 3 Baños,290000.0,MultiFamiliar,"Barrio-Coco Nuevo , Salinas",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4822064,No Broker,https://imgcache.clasificadosonline.com//FF/FS/2024/9/4/090420242045122fmgryrm.jpeg,4,3,Barrio-Coco Nuevo,Salinas,south


In [30]:
cleaned_df.to_csv("classifieds_data_v5.csv", index=False)

In [2]:
data = pd.read_csv('classifieds_data_v3.csv')

In [ ]:

data[data['region'].isna()]

In [5]:
data.to_csv("classifieds_data_v4.csv", index=False)

In [ ]:
data['Type'].unique()

In [63]:
data[data['Broker'] == 'No Broker']

,Title,Rooms,Price,Type,Location,Link,Broker,Bedrooms,Bathrooms,Neighborhood,City,region
44,NEGOCIO CON CLIENTELA OPERANDO EXITOS...,1 Baños,"$35,000",Comercial,"Urbanizacion-Sagrado Corazon , Guánica",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838405,No Broker,0,0,Urbanizacion-Sagrado Corazon,Guánica,south
52,Urb Alturas de la Placita,>= 6 Cuartos | 3 Baños,"$160,000",Casa,"Urbanizacion-Alturas De La Placita , Juncos",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838403,No Broker,6,3,Urbanizacion-Alturas De La Placita,Juncos,east
75,Solar 5 minutos de en Boquerón 1265 m...,NaN,"$88,000",Solar,"Barrio-Boqueron , Cabo Rojo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838396,No Broker,0,0,Barrio-Boqueron,Cabo Rojo,west
80,"UNICA residencia, Toa Alta - DtMF",5 Cuartos | > Baños,"$695,000",Casa,"Urbanizacion-Haciendas El Paraiso , Toa Alta",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838397,No Broker,0,0,Urbanizacion-Haciendas El Paraiso,Toa Alta,north
83,"Casa, Boquerón, 3 cuartos, 1 Baño 295...",3 Cuartos | 1 Baños,"$295,000",Casa,"Urbanizacion-Lumar , Cabo Rojo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838395,No Broker,3,1,Urbanizacion-Lumar,Cabo Rojo,west
